In [8]:
import os
import warnings

import numpy

import pandas as pd

from plotnine import *

import alignparse.ccs
import alignparse.consensus
import alignparse.minimap2
import alignparse.targets
from alignparse.constants import CBPALETTE

import dms_variants.plotnine_themes
import dms_variants.utils

In [9]:
warnings.simplefilter('ignore')

In [10]:
theme_set(dms_variants.plotnine_themes.theme_graygrid())


In [11]:
outdir = './output_files/'
os.makedirs(outdir, exist_ok=True)

In [22]:
targetfile = 'data/flu_sequences/pacbio_amplicons/fluPB2_amplicons.gb'

nlines_to_show = 85
with open(targetfile) as f:
    print(''.join(next(f) for _ in range(nlines_to_show)))

LOCUS       	fluPB2mid                2432 bp	DNA
DEFINITION  	fluPB2mid.
ACCESSION   	fluPB2mid
VERSION     	fluPB2mid
KEYWORDS    .
SOURCE      	.
  ORGANISM  	.
		      	.
FEATURES             Location/Qualifiers
     sequenced_mRNA_1	1..1182
     termini5	1..26
     viral_tag_2	1052
     final_mRNA_nts	1179..1182
     polyA	1183..1212
     UMI 	1213..1224
     cellbarcode	1225..1240
     sequenced_mRNA_2	1264..2426
     viral_tag_1	1463
     termini3	2401..2426
ORIGIN
        1 cagctattct cagaaaggca accaggagat tgatccagtt gatagtaagc gggagagatg
       61 agcagtcaat tgctgaggca ataattgtgg ccatggtatt ctcacaggag gattgcatga
      121 tcaaggcagt taggggcgat ctgaactttg tcaatagggc aaaccagcga ctgaacccca
      181 tgcaccaact cttgaggcat ttccaaaaag atgcaaaagt gcttttccag aactggggaa
      241 ttgaatccat cgacaatgtg atgggaatga tcggaatact gcccgacatg accccaagca
      301 cggagatgtc gctgagaggg ataagagtca gcaaaatggg agtagatgaa tactccagca
      361 cggagagagt ggtagtgagt attgaccgat ttttaagggt tagagatcaa ag

In [23]:
feature_parse_specs_file = 'data/flu_sequences/pacbio_amplicons/fluCA09_features.yaml'
with open(feature_parse_specs_file) as f:
    print(f.read())

# default for genes with two variant tags
default_2tags: &default_2tags
  query_clip5: 4
  query_clip3: 4
  termini5:
    filter:
      clip5: 4
      mutation_nt_count: 1
      mutation_op_count: null
  sequenced_mRNA_1:
    filter:
      mutation_nt_count: null
      mutation_op_count: 10
    return: [mutations, accuracy]
  sequenced_mRNA_2:
    filter:
      mutation_nt_count: null
      mutation_op_count: 10
    return: [mutations, accuracy]  
  final_mRNA_nts:
    filter:
      mutation_nt_count: null
      mutation_op_count: 2
  polyA:
    filter:
      mutation_nt_count: 12
      mutation_op_count: 4
  UMI:
    return: [sequence, accuracy]
  cellbarcode:
    return: [sequence, accuracy]
  termini3:
    filter:
      clip5: 4
      mutation_nt_count: 1
      mutation_op_count: null
  variant_tag_1:
    filter:
      mutation_nt_count: 1
      mutation_op_count: null
    return: sequence
  variant_tag_2:
    filter:
      mutation_nt_count: 1
      mutation_op_count: null
    retu

In [24]:
targets = alignparse.targets.Targets(
                seqsfile=targetfile,
                feature_parse_specs=feature_parse_specs_file,
                ignore_feature_parse_specs_keys=['default_2tags'],
                allow_extra_features=True)

IndexError: string index out of range